# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#500 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846855662662                   -0.70    4.0
  2   -7.852314760936       -2.26       -1.53    1.0
  3   -7.852646077604       -3.48       -2.52    3.2
  4   -7.852646676887       -6.22       -3.35    2.2
  5   -7.852646685841       -8.05       -4.76    1.0
  6   -7.852646686727       -9.05       -5.18    4.2
  7   -7.852646686729      -11.54       -5.82    2.0
  8   -7.852646686730      -12.37       -7.30    2.0
  9   -7.852646686730      -14.57       -7.66    2.0
 10   -7.852646686730   +  -15.05       -8.60    2.0
 11   -7.852646686730   +  -14.35       -9.63    2.2
 12   -7.852646686730      -14.75       -8.52    1.0
 13   -7.852646686730      -14.45       -8.44    1.0
 14   -7.852646686730   +  -14.57       -8.64    1.0
 15   -7.852646686730      -14.57       -8.30    1.0
 16   -7.852646686730   +  -14.75       -8.08    1.8
 17   -7.852646686730   +  -14.57       -8.87 

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846782781363                   -0.70           3.8
  2   -7.852526125012       -2.24       -1.62   0.80    5.0
  3   -7.852610521331       -4.07       -2.77   0.80    5.2
  4   -7.852646568441       -4.44       -3.58   0.80    7.2
  5   -7.852646683509       -6.94       -4.57   0.80    9.0
  6   -7.852646686681       -8.50       -5.11   0.80   10.5
  7   -7.852646686725      -10.35       -6.15   0.80   11.5
  8   -7.852646686730      -11.30       -7.27   0.80   13.2
  9   -7.852646686730      -13.37       -7.67   0.80   15.2
 10   -7.852646686730   +  -15.05       -9.26   0.80   15.8
 11   -7.852646686730   +    -Inf       -9.32   0.80   18.5
 12   -7.852646686730      -14.75      -10.17   0.80   18.8
 13   -7.852646686730   +  -15.05      -11.40   0.80   20.5
 14   -7.852646686730   +    -Inf      -11.82   0.80   23.0
 15   -7.852646686730   +  -15.05      -

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.405134e+01     3.399705e+00
 * time: 0.486814022064209
     1     1.383154e+00     1.866079e+00
 * time: 0.7477850914001465
     2    -1.520167e+00     1.945090e+00
 * time: 0.7741661071777344
     3    -3.750481e+00     1.816977e+00
 * time: 0.8120529651641846
     4    -4.999659e+00     1.575568e+00
 * time: 0.8494269847869873
     5    -6.673130e+00     9.642903e-01
 * time: 0.887239933013916
     6    -7.318201e+00     4.301936e-01
 * time: 1.0013301372528076
     7    -7.616303e+00     2.143643e-01
 * time: 1.0265381336212158
     8    -7.759562e+00     1.779643e-01
 * time: 1.0532031059265137
     9    -7.804393e+00     9.443377e-02
 * time: 1.0802600383758545
    10    -7.828219e+00     6.800067e-02
 * time: 1.1071290969848633
    11    -7.842270e+00     4.624946e-02
 * time: 1.1339831352233887
    12    -7.848170e+00     3.134795e-02
 * time: 1.161303997039795
    13    -7.850821e+00     1.795550e-02
 * time: 1.1884660720825

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846876062124                   -0.70    4.0
  2   -7.852328726675       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686708                   -2.55
  2   -7.852646686730      -10.66       -5.89
  3   -7.852646686730      -14.75      -12.59
|ρ_newton - ρ_scf|  = 1.8431649793125743e-13
|ρ_newton - ρ_scfv| = 9.198079576256594e-14
|ρ_newton - ρ_dm|   = 6.231279892159071e-10
